<a href="https://colab.research.google.com/github/markjfannon/SAI-24/blob/main/COMP3008_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: Introduction to Z3

This is a worksheet for the first COMP3008 lab.  There will be seven lab worksheets in total, each designed to be completed within one or two weeks.  It is generally expected that a student should be able to complete the exercises within the two-hour lab, however the extra lab hour and/or time at home might sometimes be required.

Working on these lab worksheets is an essential part of the module and a great opportunity to prepare for the coursework projects and exam.  We strongly advice showing your solutions to any of the teaching staff to discuss your approach and receive feedback.


### Support

You are encouraged to discuss the lab exercises with your classmates; this will not be academic misconduct.  (However, you cannot discuss individual projects with anyone except the teachers.)


## How to use Google Colab

Google Colab is a platform that allows one to run Python code in a web application.  This means that you do not need to install any software on your computer and that your code is stored in the cloud.

The easiest way to work on this lab is to make a copy of this notebook to your own Google Drive (File -> Save a copy in Drive).  **Changes that you make to this notebook will be lost once you close it!**

## Z3 Theorem Prover

The first software package that we will use in this module is called Z3 Theorem Prover.  It was developed by Microsoft and is considered to be one of the most advanced solvers in its class.  Wikipedia has an [article](https://en.wikipedia.org/wiki/Z3_Theorem_Prover) with a brief overview of Z3.

The main purpose of Z3 is software verification, i.e. verification that a program will always work correctly.  (While it is impossible to answer this question in general -- you may have heard of the halting problem -- Z3 efficiently handles very complex cases.)  The language of Z3 is tailored for such tasks.  Nevertheless, Z3 is based on the first-order logic, and this is how we will use it.

You will have to learn the relevant aspects of the language of Z3 on your own; this worksheet only explains a few basics, and the documentation on the web is scarce.  The expectation is that you will study the rest by trial and error -- which is common for such specialised software packages.


## Installation of the Python package

Run the below command once, to install the Microsoft Z3 package.  This may take 20-30 seconds.

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 18.0 MB/s eta 0:00:00


Z3 Solver accepts knowledge bases in the SMT2 format.  In this lab, you will use the SMT2 format to encode some simple knowledge.  You do not need to understand the Python code below; its only function is to pass the SMT2 program to the Z3 solver and then run Z3 to check if the program is satisfiable and print the model.

In [2]:
smt2program = """
(declare-const A Bool)
(declare-const B Bool)
(assert (and (xor A B) A))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[A = True, B = False]


## Exercises

1. Spend a minute to carefully read the above program (the content of the `smt2program` variable).  It is in the so-called parenthesised prefix notation, known for being used in LISP -- one of the first functional programming languages.  In this notation, each pair of parentheses is a function call, with the first word in the parentheses being the function name and the rest of the items being parameters.  Pay attention to the third line (the `assert' command); can you recognise the FOL formula encoded there?

2. Run the code snippet above.  The output tells you that the program is _satisfiable_, and also gives the values of variables $A$ and $B$ that satisfies it -- a model.  Check that these values satisfy the formula encoded in line 3 of the SMT2 program.

3. Read the following very brief reference for the Z3 language.

 To comment out a line, place semicolon at the beginning of it:
 ```
 ; Z3 will ignore this line
 ```
 Every command in Z3 is a function call and has the following format:
```
(<function name> <parameter 1> <parameter 2> ...)
```
 Each parameter can also be a function call.

 A few functions you may need:

Function| Description
--------|-----------
`and`   | Logical and; can take more than two parameters
`or`    | Logical or; can take more than two parameters
`not`   | Logical not
`=>`    | Implication ($\rightarrow$)
`=`     | Equivalence ($\leftrightarrow$)
`declare-const` | Declares a constant; first parameter is the constant name; second parameter is the constant type
`assert` | Takes one parameter and demands that it evaluates to true


4. Translate each of the following formulas into the SMT2 language and use Z3 to find the values of the constants that satisfy each of them (edit the `smt2program` content).  Check the answers by substituting them into the original formulas.

 1. $A \lor \lnot B$
 1. $(A \lor \lnot B \lor \lnot C) \land B \land C$
 1. $((A \rightarrow B) \leftrightarrow (A \land B)) \land \lnot B$
 1. $((A \land \lnot B) \lor (\lnot A \land B)) \land (A \leftrightarrow B) \land \lnot (C \leftrightarrow B)$

 (Note that not all these formulas are satisfiable.)

In [13]:
smt2program = """
(declare-const A Bool)
(declare-const B Bool)
(declare-const C Bool)
; Question 1 (assert (and A (not B)))
; Question 2 (assert (and (or A (not B) (not C)) B C))
; Question 3 (assert (and (= (=> A B) (and A B)) (not B)))
; Question 4 (assert (and  (or (and A (not B)) (and (not A) B)) (= A B) (not (= C B))))
; All questions are satisfiable except 4
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[]


5. Z3 can handle non-Boolean variables.  For example, it can handle integers; just use `Int` instead of `Bool` when defining a constant.  The supported functions are `+`, `-`, `*`, `<`, `>`, etc.

 Check the following example of a program.  It defines two integer constants and requests that $x + y < 10$ and $x > 8$.  Note that we can write multiple `assert` commands to demand multiple formulas to be satisfied.


In [14]:
smt2program = """
(declare-const x Int)
(declare-const y Int)
(assert
    (<
        (+ x y)
        10
    )
)
(assert (> x 8 ))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[x = 9, y = 0]


6. Using Z3, find a solution to the following mathematical problem:
Find a number $x$ that is greater than 1 and that is divisible by 3 and 7.  
Hint: you do not need any new commands/functions.

In [16]:
smt2program = """
(declare-const x Int)
(assert (> x 1))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[x = 2]


7. Using Z3, find a solution to the following mathematical problem.  There is a monotonically-increasing sequence of three integers from the interval $[1, 7]$; in other words, each next number is strictly greater than the previous number.
 We know that the first and the second numbers are even (divisible by 2).  We also know that the third number is the sum of the first and the second numbers.
  What are the numbers in the sequence?

In [ ]:
smt2program = """
???
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())